<img src="../../shared/img/banner.svg"></img>

# Lab 01 - Visualize, Munge, Repeat

**REMEMBER**: if you downloaded this lab to datahub with an interact link,
it will be called `lab01_blank.ipynb`.

Before doing any work, go to the dropdown menu, `File > Make a Copy`,
and then rename that copy to `lab01.ipynb`.
Work in the copy, rather than the original.

In [ ]:
%matplotlib inline

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed
from shared.src import style

In [ ]:
from pathlib import Path
import random

from client.api.notebook import Notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import shared.src.utils.util as util
import utils.plot

sns.set_context(context="notebook", font_scale=1.7)

In [ ]:
ok = Notebook("ok/config")

## Learning Objectives

1. Get hands-on experience with real data using `pandas`.
1. Use `seaborn` to create high-quality visualizations of that data quickly.
1. Recognize the difficulties in working with real data.

Even though we've only covered a bit of Python and pandas and no statistics,
we're already about half-way to becoming data scientists!

Every year, the company Figure Eight produces a
[data science report](https://www.figure-eight.com/download-2017-data-scientist-report/),
based off of a survey of data scientists
(in the case of the numbers below 187 of them, in the year 2017).
For the price of your contact information,
you can read the report and learn some neat tidbits about the state of data science
(within the last few years).

One headline-grabber was their note that
the process of "collecting, labeling, cleaning, and organizing data",
commonly known as _data mungeing_, took up on average 53% of the respondents' time.
Note that only around 15% of them listed this activity as one of their three favorites,
among competitors like "building and modeling data" and "refining algorithms".

Why does mungeing take up so much time?

For one, the manner in which data is collected often dictates its form,
rather than what would be convenient for downstream programmers.
Things like units, data types, and conventions need careful attention.

For another, real data is often far more complicated than what we work with in classes,
especially math classes and especially data "in the wild",
as opposed to data collected in a controlled research experiment.
Measurements can be subject to outliers, missing values, and other complexities.

In this lab, we will get some raw data and use our Python and visualization skills to munge it as best we can.

## Loading and Viewing the Data

The data for the lab this week is a collection of water temperature and quality measurements
from a number of beaches in Chicago during the summer for a few recent years.
It was downloaded from
[data.world](https://data.world),
a repository of publicly-available datasets,
and is originally from the
[City of Chicago's Open Data Portal](https://data.cityofchicago.org/).

More information about the data is
[here](https://data.cityofchicago.org/Parks-Recreation/Beach-Water-Quality-Automated-Sensors/qmqz-2xku).

The cell below will load the data in from its `.csv` file.

In [ ]:
beach_raw = pd.read_csv(Path("data") / "beach-water-quality-automated-sensors-1.csv")

This is the raw data. As much as possible, we want to keep this entirely pristine, rather than editing it directly.

We'll instead work primarily in a different dataframe, `beach`, that we'll build up programmatically through the lab.

In [ ]:
beach = pd.DataFrame()

## Setting a `name` Column

The first thing to do with any new dataframe is to look at the columns and the first few entries, with `.head()`,

In [ ]:
beach_raw.head()

Each of our observations seems to come from a different beach, given by the column `Beach Name`.
Whatever downstream analysis we do, we'll want to compare across beaches.

Notice that every beach name ends in `"Beach"`.
This is redundant information, so let's remove it for the version in the `beach` dataframe.

In a cell below, create a column in `beach` called `name` (no caps)
that has the contents of `Beach Name`,
but without that pesky `" Beach"` at the end.
You can either do this with indexing (how many letters does `" Beach"` have?) or with string tools,
if you're familiar with them.
Don't forget to remove the space!

In [ ]:
ok.grade("q1")


## Missing Values: Null

After looking at the `head` to see the columns and a few of their values,
it's a good idea to use `.sample()` to take a look at some random values of each column.

In [ ]:
beach_raw.sample(10)

One striking fact about most samples is that the `Transducer Depth` variable has a large number of `N`ot `a` `N`umber, or `NaN` values.
This value, `np.nan`, is used, along with a few others, to represent missing data in pandas.
You can check if a value is a "missing data" value
with the function `pd.isnull`.

In [ ]:
[pd.isnull(np.nan), pd.isnull(None)]

In [ ]:
[pd.isnull(0), pd.isnull(False), pd.isnull(-99), pd.isnull(-np.inf)]

Running `.sample` repeatedly would give us a sense for how common `NaN` values are,
but there's a better way:
we can use `.apply` with the function `pd.isnull`,
then compute the average with `.mean`.
`True` becomes `1` and `False` becomes `0`,
so the result is the fraction of values that are null and therefore missing.

In a cell below,
compute the fraction of rows in the `Transducer Depth` column that contain null values,
then print your result.
The code snippet below might be helpful.

```python
depth_null_frac = ?[?].apply(pd.isnull).mean()
print(depth_null_frac)
```

In [ ]:
ok.grade("q2")

That's a high fraction!
If this were one of the variables we really wanted to use in our analysis,
we'd be in trouble.
We'd need to find a new source of data.

Let's presume this column isn't important to us, and just not include it in `beach`.

## Missing Values: Non-Null

Just because data is not null according to pandas doesn't mean it's not missing.
Depending on the context, other data values can mean "missing", like:
- `"Null"` (the string)
- the number `0`
- `"N/A"`
- the number `-99`, for some reason

But pandas doesn't want to presume any of that context, so we have to search for that sort of thing ourselves.

These values will often show up as "outliers", or strange values,
if we try to visualize our data.

Use the code snippet below to make a plot
of the `Water Temperature` variable on the y-axis,
versus the `Beach Name` on the x-axis.
These are columns in the `beach_raw` dataframe.

A set of values should jump out at you.

```python
f, ax = plt.subplots(figsize=(16, 4))
sns.stripplot(x=?, y=?, data=?, jitter=True);
# plt.xticks(rotation=15);  # uncomment this line if the labels are smushed together
```

The values in the `"Water Temperature"` column are the measured temperatures of the water at the beach, in Celsius.
Zero Celsius is the freezing point of water, and is a quite unlikely number for the temperature at a beach during a Chicago summer.
This must be either a sensor failure or other kind of missing value.

In some cells below, make a "cleaned" version of the `Water Temperature` column by applying the function below, `zero_to_nan`, that takes `0` values and turns them into `np.nan`s.
Add it to the `beach` dataframe as the column `water_temp`.

Then, recreate the `stripplot` from above, but using the `water_temp` and `name` columns from `beach` instead.

In [ ]:
def zero_to_nan(value):
    if value == 0:
        return np.nan
    else:
        return value

## Data Transforms

Some data is measured in a form that's not very suitable for statistical analysis.

The information in the `Turbidity` column represents the "cloudiness" of the water,
measured in
[Nephelometric Turbidity Units](https://en.wikipedia.org/wiki/Turbidity#Measurement).

Run the cell below to
create a `distplot` (histogram + smoothed histogram)
of the `Turbidity` column of `beach_raw`.

In [ ]:
sns.distplot(beach_raw["Turbidity"].dropna());

The vast majority of the points have very low turbidity:
the water is very clear.

But a few points have very high turbidity:
perhaps a solid object was sucked into the nephelometer.

It will be difficult to work with data that varies over this many orders of magnitude.

The usual "trick" is to apply a _logarithm_ function,
`np.log10` in this case.
Logarithms turn orders of magnitude into something more like normal numbers:
if a value `a` is 10 times bigger than a value `b`,
`np.log10(a) - np.log10(b)` is `1`
(try it!).
The resulting data usually looks "nicer",
as you'll see below.

First, apply `zero_to_nan` to `beach_raw["Turbidity"]` and make that a column in `beach` called `turbidity`.
Zeros would represent infinitely clear water, so we'll treat that as missing.

Then, apply `np.log10` to that column, making a new column called `log10_turbidity`.
Finally, again make a `distplot` (refer to the code cell above if you need a template).

After the transformation, the data is more evenly spread across its possible values,
which no longer cross such a wide range.

In [ ]:
ok.grade("q3")

## Visualizing Continuous Relationships Between Numerical Columns

Now, we're ready to try one of our first really interesting visualizations:
is there a relationship betwen the temperature and the (log-transformed) turbidity?

Use the code snippet below to compare the `water_temp` and `log10_turbidity` columns of `beach`.

```python
sns.jointplot(?, ?, data=?, alpha=0.01, stat_func=None, size=8);
```

Nothing immediately interesting jumps out of this plot to me. Maybe if we split the data up more?

Let's bring in our `.name` column, and plot the scatterplot in the middle on a per-beach basis using `lmplot`,
based on the code snippet below.

```python
sns.lmplot(?, ?, data=?, fit_reg=False,
           hue="name", col="name", col_wrap=3,
           scatter_kws={"alpha": 0.05});
```

Now some interesting patterns are appearing:
something irregular is happening at Montrose Beach,
which has abnormally low turbidities,
and at Ohio Street Beach, which has abormally high _and_ low turbidites, but only for one temperature.

If we were continuing this analysis,
we might want to look deeper:
are those abnormal values occurring at specific times,
for example?

But to do that, we'll need to work with the time data.

## Working with Times

The time information in our data is in the column `"Measurement Timestamp"`,
where it's represented as a string.

The pandas function `pd.to_datetime` can convert a string that represents a time into
a more manageable Python object, a `DateTime`.

In a cell below, use `pd.todatetime` to convert `"Measurement Timestamp"` into a column of `DateTime`s,
and then add it to `beach` as a column called `"time"`.

Then, pass `beach` to `utils.plot.temp_over_time` to plot the `water_temp` column over time.

## Defining Categories

Even though time is in general continuous,
the plot above makes it clear that the times _in our data_ fall into discrete categories.

When that's the case, it's helpful to define those categories as explicit values
and put the information in a separate column.

In a cell below, apply the function `to_year` to the `"time"` column
and make it a column of `beach` called `"year"`.
See how easy it is to get information like the year from a `DateTime`,
as opposed to a string!

In [ ]:
def to_year(dt):
    return dt.year

Then, use the code snippet below to make a `barplot`
with the `year` on the x-axis and the `water_temp` on the y-axis.

```python
f, ax = plt.subplots(figsize=(16, 6))
sns.barplot(x=?, y=?, hue="name", data=?);
```

In [ ]:
ok.grade("q4")

## Removing Data

Notice that several beaches don't have any temperature data for 2016 or 2017,
and all appear to have only one data point apiece for 2013.

For some analyses, it's important to use measurements where you have values
for every category.
We'll practice working with this case by using a selector,
or `Bool`ean `Series`,
to select only those rows of `beach`
that come from the years 2014 and 2015.

We'd like to remove that data but still leave `beach` intact
so that our colleagues, using our notebook later,
can work with either, depending on their needs.

So take the selected rows of `beach` and turn them into a new dataframe,
`clean_beach`.
This final dataframe
should have only rows from 2014 and 2015,
but all of the columns from `beach`.

In [ ]:
ok.grade("q5")

## Visualizing Relationships Between Numerical and Categorical Columns

The `clean_beach` dataframe is now in the perfect format for visualizing differences across years for each of our beaches.

Using the code snippet below,
make a `pointplot` of the data in `clean_beach`,
again with `year` on the x-axis and
`water_temp` on the y-axis.

```python
f, ax = plt.subplots(figsize=(12, 12))
sns.pointplot(x=?, y=?, hue="name", data=?);
```

Now some interesting patterns are starting to emerge:
some of the beaches got colder, while others got warmer, or didn't change.

Later on in this course, we will learn how to analyze the differences in this plot with a traditional stats model,
called an _ANOVA_,
and with less traditional models, which have no names.

In [ ]:
ok.score()